Image Segmentation use: https://docs.ultralytics.com/modes/predict/#inference-sources

For model Training use: https://github.com/computervisioneng/image-classification-yolov8/tree/main/local_env

Hyper parameter tuning: https://docs.ultralytics.com/guides/hyperparameter-tuning/


## Importing Libraries

Importing the necessary libraries for the project

And initializing constants

```python

```


In [ ]:
!python -m venv venv

In [ ]:
!python -m pip install --upgrade pip

In [2]:
%pip install opencv-python-headless numpy ultralytics Pillow ipywidgets wandb
%pip install ultralytics "ray[tune]"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [17]:
import cv2
import numpy as np
import os
from ultralytics import YOLO
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance, ImageOps
import wandb

In [19]:
### CONSTANTS
# Image properties
IMAGE_WIDTH = 640
IMAGE_HEIGHT = 480
IMAGE_CHANNELS = 3

IMAGE_SIZE = (IMAGE_WIDTH, IMAGE_HEIGHT)

IMAGE_EXT = ('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff', '.gif')


# Director paths
PROCESSED_DS_PATH_NM = './processed_dataset/NoMask'
PROCESSED_DS_PATH_M = './processed_dataset/Mask'
RAW_DS_PATH = './Data'

## MODELS
# Segmentation
SEG_KWARGS = {
    'classes':  0,
    'imgsz': IMAGE_WIDTH,
    'boxes':True,
    'save':True,
    'show_labels':False,
    'show_conf':False,
    'max_det':1,
}
# Classification
CLASS_KWARGS = {
    'epochs': 100,
    'project': 'models',
}
MODEL_KWARGS = {
    'save': True,
    'project': './',
    'name': 'tests',
}

## Preparig data for training


### Preparing Models

```python

```


In [3]:
img_seg_model = YOLO('yolov8n-seg.pt')
img_class_model = YOLO("yolov8s-cls.pt")

### Preparing dataset

```python

```


In [4]:
def process_images(
        source_folder=RAW_DS_PATH, 
        destination_folder=PROCESSED_DS_PATH_NM, 
        grayscale=True, 
        resize=True,
        resize_dimensions=IMAGE_SIZE):
    if not os.path.exists(source_folder):
        print(f"Source folder does not exist: {source_folder}")
        return
    if os.path.exists(destination_folder):
        print(f"Destination folder already exists: {destination_folder}")
        return

    os.makedirs(destination_folder, exist_ok=True)

    for subdir, dirs, files in os.walk(source_folder):
        for file in files:
            if file.lower().endswith(IMAGE_EXT):
                source_path = os.path.join(subdir, file)
                relative_path = os.path.relpath(subdir, source_folder)
                destination_subdir = os.path.join(destination_folder, relative_path)
                os.makedirs(destination_subdir, exist_ok=True)

                destination_path = os.path.join(destination_subdir, file)

                image = cv2.imread(source_path)
                if image is None:
                    print(f"Unable to read image: {source_path}")
                    continue

                if grayscale:
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

                if resize:
                    image = cv2.resize(image, resize_dimensions)

                cv2.imwrite(destination_path, image)

In [5]:
def mask_images(
        source_folder = PROCESSED_DS_PATH_NM,
        destination_folder=PROCESSED_DS_PATH_M):
    
    if not os.path.exists(source_folder):
        print(f"Source folder does not exist: {source_folder}")
        return
    if os.path.exists(destination_folder):
        print(f"Destination folder already exists: {destination_folder}")
        return

    root_path_length = len(source_folder) + len(os.path.sep)

    # Set comprehension to get only directories at depth 2
    subdirs = {subdir[root_path_length:] for subdir, dirs, files in os.walk(source_folder)
               if subdir.count(os.path.sep) == 2}
    
    for subdir in subdirs:
        img_folder = os.path.join(source_folder, subdir)
        print(img_folder)
        img_seg_model(img_folder, project=destination_folder, name=subdir, **SEG_KWARGS)
    

In [ ]:
process_images()

mask_images()

## Training

```python

```


In [6]:
WANDB_API_KEY = '4fe204d9d24026480f7e9f42423087b20279ef32'

In [9]:
%wandb login 4fe204d9d24026480f7e9f42423087b20279ef32

UsageError: unrecognized arguments: 4fe204d9d24026480f7e9f42423087b20279ef32


In [ ]:
img_class_model.tune(data=PROCESSED_DS_PATH_M, iterations=10)

In [ ]:
%pip install wandb

In [11]:
%wandb disabled

In [20]:
wandb.init(mode='disabled');

try:

    run = max(os.listdir(CLASS_KWARGS['project']))

    run = 1 + int(run)

except:

    run = 1


img_class_model.train(data=PROCESSED_DS_PATH_M, name=run, **CLASS_KWARGS)

New https://pypi.org/project/ultralytics/8.0.209 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.208  Python-3.8.10 torch-2.1.0+cpu CPU (AMD Ryzen 5 5600 6-Core Processor)
engine\trainer: task=classify, mode=train, model=yolov8s-cls.pt, data=./processed_dataset/Mask, epochs=100, patience=50, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=0, project=models, name=1014, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, aug

ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002718F2D8A60>
curves: []
curves_results: []
fitness: 0.9403973519802094
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.887417197227478, 'metrics/accuracy_top5': 0.9933775067329407, 'fitness': 0.9403973519802094}
save_dir: WindowsPath('models/1014')
speed: {'preprocess': 0.0, 'inference': 9.158846558324548, 'loss': 0.0, 'postprocess': 0.0}
task: 'classify'
top1: 0.887417197227478
top5: 0.9933775067329407

## Performance Evaluation

In [ ]:
cm_path = os.path.join(CLASS_KWARGS['project'], str(run), 'confusion_matrix.jpg')
results_path = os.path.join(CLASS_KWARGS['project'], str(run), 'results.jpg')

cm_img = plt.imread(cm_path)
results_img = plt.imread(results_path)

plt.imshow(cm_img)
plt.show()

plt.imshow(results_img)
plt.show()

## Prediction


In [35]:
def get_model(run, best=False):
    weight = 'weights/best.pt' if best else 'weights/last.pt'
    path = os.path.join(CLASS_KWARGS['project'], str(run), weight)
    return YOLO(path)

In [36]:
def predict(image_path):
    try:
        runs = os.listdir(CLASS_KWARGS['project'])
    except:
        print("No runs found")
        return
    i = input(f"Choose a run from {runs}: ")

    while i not in runs:
        i = input(f"Choose a run from {runs}: ")
    
    model = get_model(i)

    model(image_path, **MODEL_KWARGS)

In [ ]:

predict('./processed_dataset/Mask/val/6/')



In [ ]:
#process_images(source_folder='./Data/datanew', destination_folder='./Data/datanewresult/')
mask_images(source_folder='./Data/datanewresult', destination_folder='./Data/datanewresultmask/')